In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
disease_data = pd.read_csv("/content/parkinsons.csv")

In [ ]:
disease_data.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [ ]:
disease_data.shape

(195, 24)

In [ ]:
disease_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [ ]:
disease_data.isnull().sum()

name                0
MDVP:Fo(Hz)         0
MDVP:Fhi(Hz)        0
MDVP:Flo(Hz)        0
MDVP:Jitter(%)      0
MDVP:Jitter(Abs)    0
MDVP:RAP            0
MDVP:PPQ            0
Jitter:DDP          0
MDVP:Shimmer        0
MDVP:Shimmer(dB)    0
Shimmer:APQ3        0
Shimmer:APQ5        0
MDVP:APQ            0
Shimmer:DDA         0
NHR                 0
HNR                 0
status              0
RPDE                0
DFA                 0
spread1             0
spread2             0
D2                  0
PPE                 0
dtype: int64

In [ ]:
disease_data.describe()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
count,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,...,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000
mean,154.228641,197.104918,116.324631,0.006220,0.000044,0.003306,0.003446,0.009920,0.029709,0.282251,...,0.046993,0.024847,21.885974,0.753846,0.498536,0.718099,-5.684397,0.226510,2.381826,0.206552
std,41.390065,91.491548,43.521413,0.004848,0.000035,0.002968,0.002759,0.008903,0.018857,0.194877,...,0.030459,0.040418,4.425764,0.431878,0.103942,0.055336,1.090208,0.083406,0.382799,0.090119
min,88.333000,102.145000,65.476000,0.001680,0.000007,0.000680,0.000920,0.002040,0.009540,0.085000,...,0.013640,0.000650,8.441000,0.000000,0.256570,0.574282,-7.964984,0.006274,1.423287,0.044539
25%,117.572000,134.862500,84.291000,0.003460,0.000020,0.001660,0.001860,0.004985,0.016505,0.148500,...,0.024735,0.005925,19.198000,1.000000,0.421306,0.674758,-6.450096,0.174351,2.099125,0.137451
50%,148.790000,175.829000,104.315000,0.004940,0.000030,0.002500,0.002690,0.007490,0.022970,0.221000,...,0.038360,0.011660,22.085000,1.000000,0.495954,0.722254,-5.720868,0.218885,2.361532,0.194052
75%,182.769000,224.205500,140.018500,0.007365,0.000060,0.003835,0.003955,0.011505,0.037885,0.350000,...,0.060795,0.025640,25.075500,1.000000,0.587562,0.761881,-5.046192,0.279234,2.636456,0.252980
max,260.105000,592.030000,239.170000,0.033160,0.000260,0.021440,0.019580,0.064330,0.119080,1.302000,...,0.169420,0.314820,33.047000,1.000000,0.685151,0.825288,-2.434031,0.450493,3.671155,0.527367


In [ ]:
disease_data['status'].value_counts()

status
1    147
0     48
Name: count, dtype: int64

In [ ]:
non_numeric_columns = disease_data.select_dtypes(include=['object']).columns
print(non_numeric_columns)

Index(['name'], dtype='object')


In [ ]:
for column in non_numeric_columns:
    print(disease_data[column].unique())

['phon_R01_S01_1' 'phon_R01_S01_2' 'phon_R01_S01_3' 'phon_R01_S01_4'
 'phon_R01_S01_5' 'phon_R01_S01_6' 'phon_R01_S02_1' 'phon_R01_S02_2'
 'phon_R01_S02_3' 'phon_R01_S02_4' 'phon_R01_S02_5' 'phon_R01_S02_6'
 'phon_R01_S04_1' 'phon_R01_S04_2' 'phon_R01_S04_3' 'phon_R01_S04_4'
 'phon_R01_S04_5' 'phon_R01_S04_6' 'phon_R01_S05_1' 'phon_R01_S05_2'
 'phon_R01_S05_3' 'phon_R01_S05_4' 'phon_R01_S05_5' 'phon_R01_S05_6'
 'phon_R01_S06_1' 'phon_R01_S06_2' 'phon_R01_S06_3' 'phon_R01_S06_4'
 'phon_R01_S06_5' 'phon_R01_S06_6' 'phon_R01_S07_1' 'phon_R01_S07_2'
 'phon_R01_S07_3' 'phon_R01_S07_4' 'phon_R01_S07_5' 'phon_R01_S07_6'
 'phon_R01_S08_1' 'phon_R01_S08_2' 'phon_R01_S08_3' 'phon_R01_S08_4'
 'phon_R01_S08_5' 'phon_R01_S08_6' 'phon_R01_S10_1' 'phon_R01_S10_2'
 'phon_R01_S10_3' 'phon_R01_S10_4' 'phon_R01_S10_5' 'phon_R01_S10_6'
 'phon_R01_S13_1' 'phon_R01_S13_2' 'phon_R01_S13_3' 'phon_R01_S13_4'
 'phon_R01_S13_5' 'phon_R01_S13_6' 'phon_R01_S16_1' 'phon_R01_S16_2'
 'phon_R01_S16_3' 'phon_R01_S16_4'

In [ ]:
numeric_data = disease_data.select_dtypes(include=['number'])
mean_values = numeric_data.groupby(disease_data['status']).mean()
print(mean_values)

        MDVP:Fo(Hz)  MDVP:Fhi(Hz)  MDVP:Flo(Hz)  MDVP:Jitter(%)  \
status                                                            
0        181.937771    223.636750    145.207292        0.003866   
1        145.180762    188.441463    106.893558        0.006989   

        MDVP:Jitter(Abs)  MDVP:RAP  MDVP:PPQ  Jitter:DDP  MDVP:Shimmer  \
status                                                                   
0               0.000023  0.001925  0.002056    0.005776      0.017615   
1               0.000051  0.003757  0.003900    0.011273      0.033658   

        MDVP:Shimmer(dB)  ...  Shimmer:DDA       NHR        HNR  status  \
status                    ...                                             
0               0.162958  ...     0.028511  0.011483  24.678750     0.0   
1               0.321204  ...     0.053027  0.029211  20.974048     1.0   

            RPDE       DFA   spread1   spread2        D2       PPE  
status                                                          

In [ ]:
X = disease_data.drop(columns=['name', 'status'], axis = 1)
Y = disease_data['status']

In [ ]:
X.skew().sort_values(ascending=False)

NHR                 4.220709
Jitter:DDP          3.362058
MDVP:RAP            3.360708
MDVP:Jitter(%)      3.084946
MDVP:PPQ            3.073892
MDVP:Jitter(Abs)    2.649071
MDVP:APQ            2.618047
MDVP:Fhi(Hz)        2.542146
MDVP:Shimmer(dB)    1.999389
Shimmer:APQ5        1.798697
MDVP:Shimmer        1.666480
Shimmer:DDA         1.580618
Shimmer:APQ3        1.580576
MDVP:Flo(Hz)        1.217350
PPE                 0.797491
MDVP:Fo(Hz)         0.591737
spread1             0.432139
D2                  0.430384
spread2             0.144430
DFA                -0.033214
RPDE               -0.143402
HNR                -0.514317
dtype: float64

In [ ]:
print(X)

     MDVP:Fo(Hz)  MDVP:Fhi(Hz)  MDVP:Flo(Hz)  MDVP:Jitter(%)  \
0        119.992       157.302        74.997         0.00784   
1        122.400       148.650       113.819         0.00968   
2        116.682       131.111       111.555         0.01050   
3        116.676       137.871       111.366         0.00997   
4        116.014       141.781       110.655         0.01284   
..           ...           ...           ...             ...   
190      174.188       230.978        94.261         0.00459   
191      209.516       253.017        89.488         0.00564   
192      174.688       240.005        74.287         0.01360   
193      198.764       396.961        74.904         0.00740   
194      214.289       260.277        77.973         0.00567   

     MDVP:Jitter(Abs)  MDVP:RAP  MDVP:PPQ  Jitter:DDP  MDVP:Shimmer  \
0             0.00007   0.00370   0.00554     0.01109       0.04374   
1             0.00008   0.00465   0.00696     0.01394       0.06134   
2             0.00

In [ ]:
print(Y)

0      1
1      1
2      1
3      1
4      1
      ..
190    0
191    0
192    0
193    0
194    0
Name: status, Length: 195, dtype: int64


In [ ]:
validation_size = 0.2
seed = 7

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size=validation_size, random_state=seed)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(195, 22) (156, 22) (39, 22)


In [ ]:
#Number of samples in each class before resampling
#1 for Parkinson disease
#0 for healthy
print("Original training data distribution:")
print(Y_train.value_counts())

Original training data distribution:
status
1    115
0     41
Name: count, dtype: int64


In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X_train)

StandardScaler()

In [ ]:
X_train = scaler.transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
print(X_train)

[[-0.16819857  0.22629182 -0.83791837 ... -0.34570641 -0.09091189
  -0.44537292]
 [ 0.7757276   0.08902002  1.29455556 ... -0.2861069   0.06380775
  -0.79700594]
 [-0.9990015  -0.80030161 -0.34048128 ...  0.96627053  0.18057631
   0.05524759]
 ...
 [-0.40529647 -0.30011056 -1.18795762 ... -0.23161627 -0.14945383
   0.24009742]
 [-1.18719615  0.19253974 -0.92177382 ...  0.14072248  1.2844657
   0.16712639]
 [-0.923594   -0.84407052 -0.2394943  ... -0.32379079 -0.96797356
  -0.21704176]]


**Without Data Augmentation**

In [ ]:
clf_without_aug = svm.SVC(kernel='linear')

In [ ]:
clf_without_aug.fit(X_train, Y_train)

SVC(kernel='linear')

In [ ]:
num_folds = 10
scores_without_aug = cross_val_score(clf_without_aug, X_train, Y_train, cv=num_folds)
print("Cross-validation scores:", scores_without_aug)
print("Mean cross-validation score:", scores_without_aug.mean())

Cross-validation scores: [0.75       0.8125     0.75       0.9375     0.8125     0.75
 1.         0.93333333 0.73333333 0.8       ]
Mean cross-validation score: 0.8279166666666666


In [ ]:
#Evaluation on training data
X_train_prediction_without_aug = clf_without_aug.predict(X_train)
training_data_accuracy_without_aug = accuracy_score(Y_train, X_train_prediction_without_aug)
print("Training data accuracy:", training_data_accuracy_without_aug)

Training data accuracy: 0.9166666666666666


In [ ]:
#Evaluation on training data
X_train_prediction_without_aug_2 = clf_without_aug.predict(X_train)
print("Training Classification Report:")
print(classification_report(Y_train, X_train_prediction_without_aug_2))

Training Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.73      0.82        41
           1       0.91      0.98      0.95       115

    accuracy                           0.92       156
   macro avg       0.92      0.86      0.88       156
weighted avg       0.92      0.92      0.91       156



In [ ]:
# Evaluate on test data
X_test_prediction = clf_without_aug.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print("Test data accuracy:", test_data_accuracy)

Test data accuracy: 0.8974358974358975


In [ ]:
# Evaluate on test data
X_test_prediction_no_model_aug_case = clf_without_aug.predict(X_test)
print("Test Classification Report:")
print(classification_report(Y_test, X_test_prediction_no_model_aug_case))

Test Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.57      0.67         7
           1       0.91      0.97      0.94        32

    accuracy                           0.90        39
   macro avg       0.86      0.77      0.80        39
weighted avg       0.89      0.90      0.89        39



**Adding Noise Only**

In [ ]:
#Define the noise level (standard deviation of the noise)
noise_level = 0.01

In [ ]:
# Add Gaussian noise to the training data
noise = np.random.normal(0, noise_level, X_train.shape)
X_train_noisy = X_train + noise

In [ ]:
# Combine original and noisy data
X_train_augmented = np.vstack((X_train, X_train_noisy))
Y_train_augmented = np.hstack((Y_train, Y_train))

In [ ]:
# Train the SVM model
clf_only_noisy = svm.SVC(kernel='linear')
clf_only_noisy.fit(X_train_augmented, Y_train_augmented)

SVC(kernel='linear')

In [ ]:
num_folds = 10
only_noisy_scores = cross_val_score(clf_only_noisy, X_train_augmented, Y_train_augmented, cv=num_folds)
print("Cross-validation scores:", only_noisy_scores)
print("Mean cross-validation score:", only_noisy_scores.mean())

Cross-validation scores: [0.84375    0.84375    0.87096774 0.96774194 0.87096774 0.83870968
 0.83870968 0.87096774 0.96774194 0.87096774]
Mean cross-validation score: 0.8784274193548386


In [ ]:
#Evaluate on training data
X_train_noisy_prediction = clf_only_noisy.predict(X_train_augmented)
noisy_training_data_accuracy = accuracy_score(Y_train_augmented, X_train_noisy_prediction)
print("Training data accuracy:", noisy_training_data_accuracy)

Training data accuracy: 0.907051282051282


In [ ]:
#Evaluation on training data
X_train_noisy_prediction_2 = clf_only_noisy.predict(X_train_augmented)
print("Training Classification Report:")
print(classification_report(Y_train_augmented, X_train_noisy_prediction_2))

Training Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.76      0.81        82
           1       0.92      0.96      0.94       230

    accuracy                           0.91       312
   macro avg       0.90      0.86      0.87       312
weighted avg       0.91      0.91      0.90       312



In [ ]:
# Evaluate on test data
X_test_noisy_prediction = clf_only_noisy.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_noisy_prediction)
print("Test data accuracy:", test_data_accuracy)

Test data accuracy: 0.8974358974358975


In [ ]:
# Evaluate on test data
X_test_noisy_prediction_2 = clf_only_noisy.predict(X_test)
print("Test Classification Report:")
print(classification_report(Y_test, X_test_noisy_prediction_2))

Test Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.57      0.67         7
           1       0.91      0.97      0.94        32

    accuracy                           0.90        39
   macro avg       0.86      0.77      0.80        39
weighted avg       0.89      0.90      0.89        39



**Applying SMOTE (Synthetic Minority Over-sampling Technique )**

In [ ]:
#SMOTE
smote = SMOTE(random_state=seed)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train, Y_train)

In [ ]:
#Number of samples in each class after resampling
#1 for Parkinson disease
#0 for healthy
print("SMOTE training data distribution:")
print(Y_train_resampled.value_counts())

SMOTE training data distribution:
status
1    115
0    115
Name: count, dtype: int64


In [ ]:
clf = svm.SVC(kernel='linear')

In [ ]:
clf.fit(X_train_resampled, Y_train_resampled)

SVC(kernel='linear')

In [ ]:
print(X_train_resampled)

[[-0.16819857  0.22629182 -0.83791837 ... -0.34570641 -0.09091189
  -0.44537292]
 [ 0.7757276   0.08902002  1.29455556 ... -0.2861069   0.06380775
  -0.79700594]
 [-0.9990015  -0.80030161 -0.34048128 ...  0.96627053  0.18057631
   0.05524759]
 ...
 [ 1.1890109   0.89776436 -0.91671619 ... -0.73087693 -0.00618846
  -0.93938398]
 [ 1.11127019  0.24223729  1.63817877 ... -0.64461375 -1.04137946
  -1.52936697]
 [-0.79156694 -0.38431365 -0.22012878 ...  0.23952611 -0.70421007
  -0.35891049]]


In [ ]:
X_train_resampled.shape

(230, 22)

In [ ]:
num_folds = 10
scores = cross_val_score(clf, X_train_resampled, Y_train_resampled, cv=num_folds)
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Cross-validation scores: [0.73913043 0.7826087  0.91304348 0.69565217 0.86956522 0.73913043
 0.86956522 0.91304348 0.95652174 0.91304348]
Mean cross-validation score: 0.8391304347826087


In [ ]:
#Evaluate on training data
X_train_prediction = clf.predict(X_train_resampled)
training_data_accuracy = accuracy_score(Y_train_resampled, X_train_prediction)
print("Training data accuracy:", training_data_accuracy)

Training data accuracy: 0.8782608695652174


In [ ]:
#Evaluation on training data
X_train_prediction_smote = clf.predict(X_train_resampled)
print("Training Classification Report:")
print(classification_report(Y_train_resampled, X_train_prediction_smote))

Training Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.95      0.89       115
           1       0.94      0.81      0.87       115

    accuracy                           0.88       230
   macro avg       0.89      0.88      0.88       230
weighted avg       0.89      0.88      0.88       230



In [ ]:
# Evaluate on test data
X_test_prediction = clf.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print("Test data accuracy:", test_data_accuracy)

Test data accuracy: 0.8717948717948718


In [ ]:
# Evaluate on test data
X_test_prediction_smote_case = clf.predict(X_test)
print("Test Classification Report:")
print(classification_report(Y_test, X_test_prediction_smote_case))

Test Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.71      0.67         7
           1       0.94      0.91      0.92        32

    accuracy                           0.87        39
   macro avg       0.78      0.81      0.79        39
weighted avg       0.88      0.87      0.88        39



**Noisy SMOTE**

In [ ]:
# Apply SMOTE on the noisy data
smote = SMOTE(random_state=seed)
X_train_noisy_resampled, Y_train_noisy_resampled = smote.fit_resample(X_train_augmented, Y_train_augmented)

In [ ]:
# Train the SVM model
clf_noisy_smote = svm.SVC(kernel='linear')
clf_noisy_smote.fit(X_train_noisy_resampled, Y_train_noisy_resampled)

SVC(kernel='linear')

In [ ]:
num_folds = 10
scores = cross_val_score(clf_noisy_smote, X_train_noisy_resampled, Y_train_noisy_resampled, cv=num_folds)
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Cross-validation scores: [0.82608696 0.91304348 0.80434783 0.84782609 0.93478261 0.7826087
 0.84782609 0.84782609 0.89130435 0.93478261]
Mean cross-validation score: 0.8630434782608696


In [ ]:
# Evaluate on training data
X_train_noisy_smote_prediction = clf_noisy_smote.predict(X_train_noisy_resampled)
training_data_noisy_smote_accuracy = accuracy_score(Y_train_noisy_resampled, X_train_noisy_smote_prediction)
print("Training data accuracy:", training_data_noisy_smote_accuracy)

Training data accuracy: 0.8891304347826087


In [ ]:
#Evaluation on training data
X_train_noisy_smote_prediction_2 = clf_noisy_smote.predict(X_train_noisy_resampled)
print("Training Classification Report:")
print(classification_report(Y_train_noisy_resampled, X_train_noisy_smote_prediction_2))

Training Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.96      0.90       230
           1       0.95      0.82      0.88       230

    accuracy                           0.89       460
   macro avg       0.90      0.89      0.89       460
weighted avg       0.90      0.89      0.89       460



In [ ]:
# Evaluate on test data
X_test_noisy_smote_prediction = clf_noisy_smote.predict(X_test)
test_data_noisy_smote_accuracy = accuracy_score(Y_test, X_test_noisy_smote_prediction)
print("Test data accuracy:", test_data_noisy_smote_accuracy)

Test data accuracy: 0.8205128205128205


In [ ]:
# Evaluate on test data
X_test_noisy_smote_prediction_2 = clf_noisy_smote.predict(X_test)
print("Test Classification Report:")
print(classification_report(Y_test, X_test_noisy_smote_prediction_2))

Test Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.71      0.59         7
           1       0.93      0.84      0.89        32

    accuracy                           0.82        39
   macro avg       0.72      0.78      0.74        39
weighted avg       0.85      0.82      0.83        39



**Applying Random Oversampling**

In [ ]:
#Applying Random oversampling
ros = RandomOverSampler(random_state=seed)
X_train_ros, Y_train_ros = ros.fit_resample(X_train, Y_train)

In [ ]:
#Number of samples in each class after resampling
#1 for Parkinson disease
#0 for healthy
print("Random Oversampling training data distribution:")
print(Y_train_ros.value_counts())

Random Oversampling training data distribution:
status
1    115
0    115
Name: count, dtype: int64


In [ ]:
clf_ros = svm.SVC(kernel='linear')

In [ ]:
clf_ros.fit(X_train_ros, Y_train_ros)

SVC(kernel='linear')

In [ ]:
num_folds = 10
scores_ros = cross_val_score(clf_ros, X_train_ros, Y_train_ros, cv=num_folds)
print("Cross-validation scores:", scores_ros)
print("Mean cross-validation score:", scores_ros.mean())

Cross-validation scores: [0.73913043 0.82608696 0.91304348 0.7826087  0.86956522 0.69565217
 0.7826087  0.82608696 0.91304348 0.91304348]
Mean cross-validation score: 0.826086956521739


In [ ]:
#Evaluate on training data
X_train_prediction_ros = clf_ros.predict(X_train_ros)
training_data_accuracy_ros = accuracy_score(Y_train_ros, X_train_prediction_ros)
print("Training data accuracy:", training_data_accuracy_ros)

Training data accuracy: 0.8608695652173913


In [ ]:
X_train_prediction_ros_2 = clf_ros.predict(X_train_ros)
print("Training Classification Report:")
print(classification_report(Y_train_ros, X_train_prediction_ros_2))

Training Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       115
           1       0.89      0.83      0.86       115

    accuracy                           0.86       230
   macro avg       0.86      0.86      0.86       230
weighted avg       0.86      0.86      0.86       230



In [ ]:
# Evaluate on test data
X_test_prediction_ros = clf_ros.predict(X_test)
test_data_accuracy_ros = accuracy_score(Y_test, X_test_prediction_ros)
print("Test data accuracy:", test_data_accuracy_ros)

Test data accuracy: 0.8461538461538461


In [ ]:
# Evaluate on test data
X_test_prediction_ros_test = clf_ros.predict(X_test)
print("Test Classification Report:")
print(classification_report(Y_test, X_test_prediction_ros_test))

Test Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.71      0.63         7
           1       0.93      0.88      0.90        32

    accuracy                           0.85        39
   macro avg       0.74      0.79      0.76        39
weighted avg       0.87      0.85      0.85        39



**ADASYN (Adaptive Synthetic Sampling)**

In [ ]:
#Applying ADASYN
adasyn = ADASYN(random_state=seed)
X_train_adasyn, Y_train_adasyn = adasyn.fit_resample(X_train, Y_train)

In [ ]:
clf_adasyn = svm.SVC(kernel='linear')
clf_adasyn.fit(X_train_adasyn, Y_train_adasyn)

SVC(kernel='linear')

In [ ]:
#Cross-validation
scores_adasyn = cross_val_score(clf_adasyn, X_train_adasyn, Y_train_adasyn, cv=num_folds)
print("ADASYN - Cross-validation scores:", scores_adasyn)
print("Mean cross-validation score:", scores_adasyn.mean())

ADASYN - Cross-validation scores: [0.7826087  0.7826087  0.91304348 0.7826087  0.86956522 0.60869565
 0.82608696 0.95652174 0.90909091 0.95454545]
Mean cross-validation score: 0.8385375494071147


In [ ]:
#Evaluate on training data
X_train_prediction_adasyn = clf_adasyn.predict(X_train_adasyn)
training_data_accuracy_adasyn = accuracy_score(Y_train_adasyn, X_train_prediction_adasyn)
print("ADASYN - Training data accuracy:", training_data_accuracy_adasyn)

ADASYN - Training data accuracy: 0.8771929824561403


In [ ]:
X_train_prediction_adasyn = clf_adasyn.predict(X_train_adasyn)
print("Training Classification Report:")
print(classification_report(Y_train_adasyn, X_train_prediction_adasyn))

Training Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.95      0.88       113
           1       0.94      0.81      0.87       115

    accuracy                           0.88       228
   macro avg       0.88      0.88      0.88       228
weighted avg       0.88      0.88      0.88       228



In [ ]:
#Evaluate on test data
X_test_prediction_adasyn = clf_adasyn.predict(X_test)
test_data_accuracy_adasyn = accuracy_score(Y_test, X_test_prediction_adasyn)
print("ADASYN - Test data accuracy:", test_data_accuracy_adasyn)

ADASYN - Test data accuracy: 0.7948717948717948


In [ ]:
# Evaluate on test data
X_test_prediction_adasyn_2 = clf_adasyn.predict(X_test)
print("Test Classification Report:")
print(classification_report(Y_test, X_test_prediction_adasyn_2))

Test Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.57      0.50         7
           1       0.90      0.84      0.87        32

    accuracy                           0.79        39
   macro avg       0.67      0.71      0.69        39
weighted avg       0.82      0.79      0.80        39

